In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/edx-courses-dataset-2021/EdX.csv
/kaggle/input/dataset-of-1200-coursera-courses/edx.csv
/kaggle/input/dataset-of-1200-coursera-courses/Barkeley_extension.csv
/kaggle/input/dataset-of-1200-coursera-courses/Oxford.csv
/kaggle/input/dataset-of-1200-coursera-courses/Stanford.csv
/kaggle/input/dataset-of-1200-coursera-courses/udacity.csv
/kaggle/input/dataset-of-1200-coursera-courses/alison.csv
/kaggle/input/dataset-of-1200-coursera-courses/MIT ocw.csv
/kaggle/input/dataset-of-1200-coursera-courses/london school of economics.csv
/kaggle/input/dataset-of-1200-coursera-courses/coursera_update.csv
/kaggle/input/dataset-of-1200-coursera-courses/pluralsight.csv
/kaggle/input/dataset-of-1200-coursera-courses/futurelearn.csv
/kaggle/input/dataset-of-1200-coursera-courses/swayam.csv
/kaggle/input/dataset-of-1200-coursera-courses/Harvard_university.csv
/kaggle/input/multi-platform-online-courses-dataset/edx.csv
/kaggle/input/multi-platform-online-courses-dataset/skillshare.csv
/kaggle/

In [54]:
# Install NLTK
!pip list | grep nltk
! pip install -U kaleido
import nltk

nltk.download('punkt')  
nltk.download('wordnet')  

# Unzip per this stackoverflow: https://stackoverflow.com/questions/73849624/getting-error-while-submitting-notebook-on-kaggle-even-after-importing-nltk-libr
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

nltk                               3.2.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.7 MB/s eta 0:00:00:00:010:01m
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [58]:
# Setup

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re

# Clean text 
def clean_text(text):
    lemma = WordNetLemmatizer() # lemmatizer
    text = re.sub("[^A-Za-z1-9 ]", "", text)
    text = text.lower()
    tokens = word_tokenize(text) # look into this tokenization
    tokens = [lemma.lemmatize(word) for word in tokens # lemmatize words and remove stopwords 
                if word not in stopwords.words("english")]
    return tokens


# Get the sentence embeddings for each course and user input with this function
# First get the word embeddings and average them out for the sentence (aka course/input)
# overall embedding

def get_document_embedding(doc, model):
    embeddings = [model.wv[word] for word in doc if word in model.wv] # Get individual embeddings into a list
    # Consider implementing exception handling 
    if len(embeddings) > 0:
        return np.mean(embeddings, axis=0) 
    else:
        return np.zeros(model.vector_size)

# Use previous functions to process user input into vector and use cosine 
# Similarity to find the most related courses
def recommend_courses(user_input, document_embeddings, data, top_n=5):
    cleaned_input = clean_text(user_input)
    input_embedding = get_document_embedding(cleaned_input, word2vec_model)
    similarities = cosine_similarity([input_embedding], document_embeddings)[0]
    top_indices = np.argsort(similarities)[-top_n:][::-1]
    recommendations = data.iloc[top_indices][['name', 'topic', 'link']]
    return recommendations

        

In [60]:
# Experiment with other data
# edx, coursera, harvard, mit ocw
# https://sparkbyexamples.com/pandas/pandas-read-multiple-csv-files/#:~:text=Load%20each%20file%20into%20individual,each%20file%20individually%20if%20needed.

# Normalize/clean course data to the name, topic, link, text format for now

dataMit = pd.read_csv("/kaggle/input/dataset-of-1200-coursera-courses/MIT ocw.csv")
dataMit.columns = map(str.lower, dataMit.columns)
dataMit.rename(columns={'name ': 'name'}, inplace=True)
dataMit = dataMit.drop(columns=['sub category', 'category', 'resource type', 'course code', 'instructure'])
dataMit.rename(columns={'course link': 'link'}, inplace=True)
dataMit['text'] = dataMit['name'] + " " + dataMit['topic'] 
dataMit = dataMit[['name', 'topic', 'link', 'text']]



dataHarvard = pd.read_csv("/kaggle/input/dataset-of-1200-coursera-courses/Harvard_university.csv")
dataHarvard.columns = map(str.lower, dataHarvard.columns)
dataHarvard.rename(columns={'link to course': 'link', 'about': 'topic'}, inplace=True)
# Remove rows where the price is not 'Free' (gpt code)
dataHarvard = dataHarvard[dataHarvard['price'] == 'Free']
# Select only the columns 'name', 'topic', and 'link'
dataHarvard = dataHarvard[['name', 'topic', 'link']]
dataHarvard['text'] = dataHarvard['name'] + " " + dataHarvard['topic'] 
print(dataHarvard.head())

# Coursera data

# Read in 2021 EdXdata
dataEdx = pd.read_csv("/kaggle/input/edx-courses-dataset-2021/EdX.csv")
dataEdx.columns = map(str.lower, dataEdx.columns)
dataEdx["topic"] = dataEdx['about'] + '. ' + dataEdx['course description']
dataEdx = dataEdx[['name', 'topic', 'link']]
dataEdx['text'] = dataEdx['name'] + " " + dataEdx["topic"]
# dataEdx.head()

# Read in 

                                                 name  \
0                 PredictionX: Lost Without Longitude   
1   Nonprofit Financial Stewardship Webinar: Intro...   
2              CS50: Introduction to Computer Science   
10           PredictionX: Omens, Oracles & Prophecies   
11             Systematic Approaches to Policy Design   

                                                topic  \
0   Explore the history of navigation, from stars ...   
1   The Introduction to Nonprofit Accounting and F...   
2   An introduction to the intellectual enterprise...   
10  An overview of divination systems, ranging fro...   
11  This free online course from Harvard Kennedy S...   

                                                 link  \
0   https://pll.harvard.edu/course/predictionx-los...   
1   https://pll.harvard.edu/course/nonprofit-finan...   
2   https://pll.harvard.edu/course/cs50-introducti...   
10  https://pll.harvard.edu/course/predictionx-ome...   
11  https://pll.harvard.edu/c

In [61]:
# Create a Combined Dataframe
data = pd.concat([dataHarvard, dataMit, dataEdx])
data.head()

,name,topic,link,text
0,PredictionX: Lost Without Longitude,"Explore the history of navigation, from stars ...",https://pll.harvard.edu/course/predictionx-los...,PredictionX: Lost Without Longitude Explore th...
1,Nonprofit Financial Stewardship Webinar: Intro...,The Introduction to Nonprofit Accounting and F...,https://pll.harvard.edu/course/nonprofit-finan...,Nonprofit Financial Stewardship Webinar: Intro...
2,CS50: Introduction to Computer Science,An introduction to the intellectual enterprise...,https://pll.harvard.edu/course/cs50-introducti...,CS50: Introduction to Computer Science An intr...
10,"PredictionX: Omens, Oracles & Prophecies","An overview of divination systems, ranging fro...",https://pll.harvard.edu/course/predictionx-ome...,"PredictionX: Omens, Oracles & Prophecies An ov..."
11,Systematic Approaches to Policy Design,This free online course from Harvard Kennedy S...,https://pll.harvard.edu/course/systematic-appr...,Systematic Approaches to Policy Design This fr...


In [62]:
# The list of tokenized sentences, ie our Corpus 
data['cleaned_text'] = data['text'].apply(clean_text) # Add clean text column to dataframe
data['cleaned_text'].head()




0     [predictionx, lost, without, longitude, explor...
1     [nonprofit, financial, stewardship, webinar, i...
2     [cs5, introduction, computer, science, introdu...
10    [predictionx, omen, oracle, prophecy, overview...
11    [systematic, approach, policy, design, free, o...
Name: cleaned_text, dtype: object

In [ ]:
# Corpus = list of tokenized sentences (already cleaned)
corpus = data['cleaned_text'].tolist()
corpus[0]

# Train Word2Vec
model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,       # Larger window for broader context
    min_count=5,    # Ignore rare words
    workers=4       # Parallelize training
)

words in corpus: 100809
corpus count: 3382


In [ ]:
# DATA exploration

vocab_len = len(model.wv)
print(f'Vocabulary size: {vocab_len}')

print('First 10 words in vocabulary:')
key_vocab = model.wv.index_to_key[:10]
print(key_vocab)

word1 = 'programming'; word2 = 'python'
similarity = model.wv.similarity(word1,word2)
print(f'\nsimilarity b/w {word1} and {word2} {round(similarity,2)}\n')

# View similar words based on gensim's model
print('Similar Words')
similar_words = {search_term: [item[0] for item in model.wv.most_similar([search_term], topn=5)]
                  for search_term in key_vocab}
similar_words

# Lower dimensionality visualisation of embeddings (100->2)
import plotly.express as px
from sklearn.manifold import TSNE
import warnings; warnings.filterwarnings('ignore')

words = sum([[k] + v for k, v in similar_words.items()], [])
wvs = model.wv[words]

tsne = TSNE(n_components=2, 
            random_state=0, 
            n_iter=10000)

X = tsne.fit_transform(wvs)
labels = words
    
fig = px.scatter(X[:, 0], X[:, 1],text=labels,
           template='plotly_white',
           width=800,
           title='Word Embedding Visualisation')
fig.show('svg',dpi=300)

In [56]:
# Get a list of the document embedding vector for each sentence in the cleaned text data. The indices will be aligned with the original course rows in dataframe
document_embeddings = [get_document_embedding(doc, model)
                      for doc in data['cleaned_text']]
print(f'list of sentence vectors/sentences: {len(document_embeddings)}')
print(f'each sentence has {document_embeddings[0].shape} dimensions')

list of sentence vectors/sentences: 3382
each sentence has (100,) dimensions


In [57]:
# User interface (abstracted away)
user_input = "Single Variable Calculus"
recommendations = recommend_courses(user_input, document_embeddings, data)
recommendations.head()

,name,topic,link
1711,History of Women in Science and Engineering,"Engineering, Science, Humanities",https://ocw.mit.edu/courses/wgs-s10-history-of...
881,Operating System Engineering,"Engineering, Computer Science, Software Design...",https://ocw.mit.edu/courses/6-828-operating-sy...
81,Introduction to Numerical Analysis for Enginee...,"Engineering, Computer Science, Mathematics",https://ocw.mit.edu/courses/2-993j-introductio...
50,Introduction to Numerical Analysis for Enginee...,"Engineering, Computer Science, Mathematics",https://ocw.mit.edu/courses/2-993j-introductio...
537,Mathematical Methods in Nanophotonics,"Engineering, Science, Physics",https://ocw.mit.edu/courses/18-369-mathematica...


# Important Notes

* Original Source: https://www.kaggle.com/code/shtrausslearning/nlp-edx-course-recommendations
* DeepSeek AI for original base which also calls the document embeddings for the input
* I have added comments to better study and understand the code as a base to build off of
* Instead of directly matching to a course index in the dataset which limits the use of the model

# Todo (to better understand and be able to present on this topic)

* Study cosine similarity\
* Word2Vec and stopwords/lemmatization